## 5-5. 소프트맥스 회귀로 MNIST 데이터 분류하기
### 1. MNIST 데이터 이해하기
숫자 0~9까지의 이미지로 구성된 손글씨 데이터셋 <br/>
총 60,000개의 훈련 데이터와 레이블, 총 10,000개의 테스트 데이터와 레이블로 구성되어져 있음. <br/>
각 이미지당 28픽셀 x 28 픽셀 = 784 픽셀이므로 784차원의 벡터로 만들어야 함

In [ ]:
for X, Y in data_loader:
    #입력 이미지를 [batch_size x 784]의 크기로 reshape
    #레이블은 원-핫 인코딩
    X = X.view(-1, 28*28)
#X는 for문에서 호출될 때 (배치크기x1x28x28)의 크기를 가지지만, view를 통해서 (배치크기x784)의 크기로 변환됨

### 2. 토치비전(torchvision) 소개하기
### 3. 분류기 구현을 위한 사전 설정

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

#GPU 연산으로
USE_CUDA = torch.cuda.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴
device = torch.device("cuda" if USE_CUDA else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다: ", device)

다음 기기로 학습합니다:  cuda


In [2]:
#랜덤시드 고정
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    
#hyperparameters
training_epochs = 15
batch_size = 100

### 4. MNIST 분류기 구현하기

In [3]:
#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [4]:
#dataset loader
#datset: 로드할 대상, batch_size: 배치크기, shuffle: 매 epoch마다 미니 배치를 셔플할 것인지 여부, drop_last: 마지막 배치를 버릴 것인지
data_loader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
# 마지막 배치를 버림으로써 다른 미니 배치보다 개수가 적은 마지막 배치를 경사 하강법에 사용하여 마지막 배치가 상대적으로 과대 평가되는 현상을 막아줌

In [5]:
#모델 설계
#MNIST data image of shape 28*28=784
linear = nn.Linear(784, 10, bias=True).to(device) #to(): 연산을 어디서 수행할지

In [6]:
# 비용함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss().to(device) #내부적으로 소프트맥스 함수를 포함하고 있음
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [7]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)
    
    for X, Y in data_loader:
        #배치 크기가 100이므로 아래의 연산에서 X는 (100, 784)의 텐서가 된다.
        X = X.view(-1, 28*28).to(device)
        #레이블은 원-핫 인코딩이 된 상태가 아니라 0~9의 정수
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print("Epoch: ", "%04d" % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
print('Learning finished')

Epoch:  0001 cost = 0.534910798
Epoch:  0002 cost = 0.359307885
Epoch:  0003 cost = 0.331088513
Epoch:  0004 cost = 0.316574335
Epoch:  0005 cost = 0.307131052
Epoch:  0006 cost = 0.300209075
Epoch:  0007 cost = 0.294897050
Epoch:  0008 cost = 0.290830195
Epoch:  0009 cost = 0.287420511
Epoch:  0010 cost = 0.284589112
Epoch:  0011 cost = 0.281816602
Epoch:  0012 cost = 0.279920727
Epoch:  0013 cost = 0.277837515
Epoch:  0014 cost = 0.276021779
Epoch:  0015 cost = 0.274444699
Learning finished
